In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/keras-model/keras/default/1/coastline_unet_model_s2_256_adapt.keras
/kaggle/input/code-file/network.py
/kaggle/input/balanced-ds/FinalDataset/images/Image_21_water_aug4_row6820_col3740.jp2
/kaggle/input/balanced-ds/FinalDataset/images/Image_15_land_aug2_row8580_col2200.jp2
/kaggle/input/balanced-ds/FinalDataset/images/Image_15_mixed_aug3_row6600_col9460.jp2
/kaggle/input/balanced-ds/FinalDataset/images/Image_10_mixed_aug1_row1540_col9900.jp2
/kaggle/input/balanced-ds/FinalDataset/images/Image_15_land_aug4_row7260_col4840.jp2
/kaggle/input/balanced-ds/FinalDataset/images/Image_1_mixed_aug0_row6160_col5500.jp2
/kaggle/input/balanced-ds/FinalDataset/images/Image_20_mixed_aug0_row2860_col0.jp2
/kaggle/input/balanced-ds/FinalDataset/images/Image_20_mixed_aug2_row7260_col9240.jp2
/kaggle/input/balanced-ds/FinalDataset/images/Image_1_mixed_aug0_row2640_col8800.jp2
/kaggle/input/balanced-ds/FinalDataset/images/Image_10_mixed_aug0_row6600_col2640.jp2
/kaggle/input/balanced-ds/Fina

In [3]:
# --- Imports ---
import tensorflow as tf
import numpy as np
import random
import os
import glob
from tqdm.notebook import tqdm
import rasterio
import math
import warnings
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

# --- 1. Setup & Configuration ---
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# --- Dataset Path ---
BASE_DATA_PATH = '/kaggle/input/balanced-ds/FinalDataset' 

IMAGES_DIR = os.path.join(BASE_DATA_PATH, 'images')
MASKS_DIR = os.path.join(BASE_DATA_PATH, 'masks')

# --- Image Dimensions ---
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 12

# --- Training Configuration ---
BATCH_SIZE = 8
BUFFER_SIZE = 1000
EPOCHS = 50
NUM_ADAPT_BATCHES = 500

# --- Model Checkpoint Path ---
MODEL_CHECKPOINT_PATH = '/kaggle/working/coastline_unet_model_s2_adapt.keras'
TENSORBOARD_LOG_DIR = '/kaggle/working/logs_coastline_s2_adapt'

# --- 2. Data Loading & Splitting ---
def get_all_file_paths(images_dir, masks_dir):
    image_files = sorted(glob.glob(os.path.join(images_dir, '*.jp2')))
    mask_files = sorted(glob.glob(os.path.join(masks_dir, '*.png')))
    
    if len(image_files) != len(mask_files):
        raise ValueError("Mismatch between number of images and masks!")
    return image_files, mask_files

print("Loading file paths...")
all_image_files, all_mask_files = get_all_file_paths(IMAGES_DIR, MASKS_DIR)

# --- Train/Validation Split (80:20) ---
print("Splitting dataset into train and validation (80:20)...")
train_img_files, val_img_files, train_mask_files, val_mask_files = train_test_split(
    all_image_files, all_mask_files, test_size=0.2, random_state=SEED
)

print(f"Training images: {len(train_img_files)}")
print(f"Validation images: {len(val_img_files)}")

# --- 3. Data Preprocessing Functions ---
@tf.function
def parse_image_mask(img_path, mask_path):
    def read_jp2(path):
        path = path.numpy().decode('utf-8')
        with rasterio.open(path) as img_ds:
            img = img_ds.read().astype(np.float32)  # (channels, H, W)
            img = np.transpose(img, (1, 2, 0))  # (H, W, channels)
            return img

    img = tf.py_function(read_jp2, [img_path], tf.float32)
    img.set_shape([IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS])

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, [IMG_HEIGHT, IMG_WIDTH], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    mask = tf.cast(mask, tf.float32)
    mask = tf.cast(mask > 127.0, tf.float32)

    return img, mask

def create_tf_dataset(image_files, mask_files, batch_size, shuffle=False, buffer_size=1000):
    dataset = tf.data.Dataset.from_tensor_slices((image_files, mask_files))
    if shuffle:
        dataset = dataset.shuffle(buffer_size)
    dataset = dataset.map(parse_image_mask, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

# --- 4. Create Datasets ---
print("Creating datasets...")
train_dataset = create_tf_dataset(train_img_files, train_mask_files, BATCH_SIZE, shuffle=True, buffer_size=BUFFER_SIZE)
val_dataset = create_tf_dataset(val_img_files, val_mask_files, BATCH_SIZE)

num_train_samples = len(train_img_files)
num_val_samples = len(val_img_files)
steps_per_epoch = math.ceil(num_train_samples / BATCH_SIZE)
validation_steps = math.ceil(num_val_samples / BATCH_SIZE)

# --- 5. Normalize the Data ---
print("Adapting normalization layer...")
normalization_layer = tf.keras.layers.Normalization(axis=-1)
adapt_dataset = train_dataset.map(lambda img, mask: img).take(NUM_ADAPT_BATCHES)
normalization_layer.adapt(adapt_dataset)

# --- 6. U-Net Model ---
def conv_block(inputs, num_filters):
    x = tf.keras.layers.Conv2D(num_filters, 3, activation="relu", kernel_initializer="he_normal", padding="same")(inputs)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Conv2D(num_filters, 3, activation="relu", kernel_initializer="he_normal", padding="same")(x)
    return x

def upsample_block(inputs, skip, num_filters):
    up = tf.keras.layers.Conv2DTranspose(num_filters, 2, strides=2, padding="same")(inputs)
    concat = tf.keras.layers.concatenate([up, skip])
    x = conv_block(concat, num_filters)
    return x

def build_unet(input_shape, norm_layer):
    inputs = tf.keras.layers.Input(input_shape)
    s = norm_layer(inputs)

    # Encoder
    c1 = conv_block(s, 16); p1 = tf.keras.layers.MaxPooling2D(2)(c1)
    c2 = conv_block(p1, 32); p2 = tf.keras.layers.MaxPooling2D(2)(c2)
    c3 = conv_block(p2, 64); p3 = tf.keras.layers.MaxPooling2D(2)(c3)
    c4 = conv_block(p3, 128); p4 = tf.keras.layers.MaxPooling2D(2)(c4)

    # Bridge
    c5 = conv_block(p4, 256)

    # Decoder
    c6 = upsample_block(c5, c4, 128)
    c7 = upsample_block(c6, c3, 64)
    c8 = upsample_block(c7, c2, 32)
    c9 = upsample_block(c8, c1, 16)

    outputs = tf.keras.layers.Conv2D(1, 1, activation='sigmoid')(c9)

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    return model

# --- Build and Compile Model ---
input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
model = build_unet(input_shape, normalization_layer)

def iou_metric(y_true, y_pred, smooth=1e-6):
    y_pred_pos = tf.cast(y_pred > 0.5, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred_pos, axis=[1,2,3])
    union = tf.reduce_sum(y_true, axis=[1,2,3]) + tf.reduce_sum(y_pred_pos, axis=[1,2,3]) - intersection
    return tf.reduce_mean((intersection + smooth) / (union + smooth))

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy", iou_metric]
)
model.summary()

# --- 7. Callbacks ---
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=15, monitor='val_iou_metric', mode='max', restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ModelCheckpoint(MODEL_CHECKPOINT_PATH, monitor='val_iou_metric', save_best_only=True, verbose=1, mode='max'),
    tf.keras.callbacks.TensorBoard(log_dir=TENSORBOARD_LOG_DIR),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7, verbose=1)
]

# --- 8. Train the Model ---
print("\nStarting training...")
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_dataset,
    validation_steps=validation_steps,
    callbacks=callbacks
)
print(f"\nTraining completed. Best model saved at: {MODEL_CHECKPOINT_PATH}")


Loading file paths...
Splitting dataset into train and validation (80:20)...
Training images: 16800
Validation images: 4200
Creating datasets...


I0000 00:00:1745944559.051171      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745944559.051840      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Adapting normalization layer...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 12)   │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization             │ (None, 256, 256, 12)   │             25 │ input_layer[0][0]      │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 256, 256, 16)   │          1,744 │ normalization[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 256, 256, 16)   │              0 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 256, 256, 16)   │          2,320 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 128, 128, 16)   │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 128, 128, 32)   │          4,640 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 128, 128, 32)   │              0 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 128, 128, 32)   │          9,248 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 64, 64, 32)     │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 64, 64, 64)     │         18,496 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 64, 64, 64)     │              0 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 64, 64, 64)     │         36,928 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 32, 32, 64)     │              0 │ conv2d_5[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 32, 32, 128)    │         73,856 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 32, 32, 128)    │              0 │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 32, 32, 128)    │        147,584 │ dropout_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3      

...

 Trainable params: 1,942,401 (7.41 MB)

 Non-trainable params: 25 (104.00 B)


Starting training...
Epoch 1/50


I0000 00:00:1745944716.534467     103 service.cc:148] XLA service 0x7fb848005210 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745944716.535336     103 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1745944716.535356     103 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1745944718.001734     103 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1745944734.263078     103 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2100/2100 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step - accuracy: 0.9144 - iou_metric: 0.5451 - loss: 0.2424
Epoch 1: val_iou_metric improved from -inf to 0.82354, saving model to /kaggle/working/coastline_unet_model_s2_adapt.keras
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 786s 361ms/step - accuracy: 0.9144 - iou_metric: 0.5451 - loss: 0.2424 - val_accuracy: 0.9291 - val_iou_metric: 0.8235 - val_loss: 0.1711 - learning_rate: 1.0000e-04
Epoch 2/50
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 0s 15us/step - accuracy: 0.0000e+00 - iou_metric: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 3/50
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step - accuracy: 0.9318 - iou_metric: 0.8002 - loss: 0.1713
Epoch 3: val_iou_metric improved from 0.82354 to 0.82939, saving model to /kaggle/working/coastline_unet_model_s2_adapt.keras
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 667s 318ms/step - accuracy: 0.9318 - iou_metric: 0.8002 - loss: 0.1713 - val_accuracy: 0.9312 - val_iou_metric: 0.8294 - val_loss: 0.1690 - learning_rate: 1.0000e